# Exercise 1: Compute pi on the GPU

0. CPU version: about 18 seconds for 512 000 000 points
1. Maximum 1024 threads (one block with 1024 threads)

In [2]:
%%time
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp311-cp311-linux_x86_64.whl size=660362 sha256=3f2e7714cd13dc3caa0a80deb17c613f4fbfeb3b413558d8bd7fac6f634cbb65
  Stored in directory: /root/.cache/pip/wheels/76/66/50/c65e6116d7e0e16abe0f7c19b50327f76724ccfefbdc61a1b9
Successfully built pycuda
CPU times: user 619 ms, sys: 79 ms, total: 698 ms
Wall time: 2min 22s


In [3]:
import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np
import time

In [4]:
rng = np.random.default_rng()

def compute_pi_cpu(n_points):
    #First, generate random points
    rng = np.random.RandomState(42)
    x_rand = rng.random(n_points)
    y_rand = rng.random(n_points)

    #Compute radius from origin
    inside = np.sqrt(x_rand**2+y_rand**2) <= 1.0
    #Count number of points inside
    n_inside = np.sum(inside)

    #n_inside = 0
    #for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2+y_rand[i]**2) <= 1.0

    #We can estimate pi by the following formula:
    #pi = 4 * n_inside / n_total
    pi = 4*n_inside/n_points

    return pi

In [5]:
tic = time.time()
print(compute_pi_cpu(1024))
toc = time.time()

print("Time to execute cpu version: {:f} seconds".format(toc-tic))

3.109375
Time to execute cpu version: 0.005301 seconds


In [6]:
pi_kernel_src = """
//Based on Stroustrup, adapted for CUDA
//pseudorandom numbers
__device__ float generateRandomNumber(long& last_draw) {
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0;
}

__global__ void computePi(unsigned int* inside, unsigned int seed) {
    //1 generate random numbers
    unsigned int tid = threadIdx.x;

    long rand_seed = seed + tid;
    float x = generateRandomNumber(rand_seed);
    float y = generateRandomNumber(rand_seed);

    //2 compute the radius from the origin
    float r = sqrt(x*x + y*y);

    //3 check if it is inside the circle
    if (r <= 1.0) { // oh-oh
        inside[tid] = 1;
    }
}
"""

mod = SourceModule(pi_kernel_src)
func = mod.get_function("computePi")

In [7]:
def compute_pi_gpu(n_points):
    bytes_per_uint = 4

    inside_gpu = cuda.mem_alloc(bytes_per_uint * n_points)

    func(inside_gpu, np.uint32(time.time()), block=(n_points,1,1), grid=(1,1,1))

    inside_cpu = np.empty(n_points, dtype=np.uint32)
    cuda.memcpy_dtoh(inside_cpu, inside_gpu)

    n_inside = np.sum(inside_cpu)

    pi = 4 * n_inside/n_points

    return pi

In [10]:
tic = time.time()
print(compute_pi_gpu(1024))
toc = time.time()

print("Time to execute gpu version: {:f} seconds".format(toc-tic))

3.15625
Time to execute gpu version: 0.001398 seconds
